# Importing Libraries

In [95]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [96]:
cur = os.getcwd()

In [97]:
df = pd.read_csv(cur+"\\data\\data.zip")
df = df.drop('Unnamed: 0', 1)

C:\Users\Anubhav\AppData\Local\Temp/ipykernel_8352/1479601220.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('Unnamed: 0', 1)


In [98]:
df

,trade Id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch
0,1274557543,43160.00,0.00056,24.169600,1646092800000,True,True
1,1274557544,43160.01,0.00052,22.443205,1646092800003,False,True
2,1274557545,43160.01,0.00182,78.551218,1646092800004,False,True
3,1274557546,43160.01,0.00631,272.339663,1646092800005,False,True
4,1274557547,43160.00,0.00229,98.836400,1646092800006,True,True
...,...,...,...,...,...,...,...
2871280,1277428823,44736.04,0.08083,3616.014113,1646233096536,False,True
2871281,1277428824,44736.04,0.08864,3965.402586,1646233096536,False,True
2871282,1277428825,44737.45,0.00535,239.345358,1646233096556,True,True
2871283,1277428826,44737.45,0.00167,74.711541,1646233096567,True,True


# Basic Logic of Market Making System

In [99]:
"""
value = (bid+ask) /2
spread = (ask-bid)+profit_target
skew = -position*risk_adjustment


buy_quote=value+skew - spread
sell_quote = value + skew + spread


In this problem,

bid = value*99.97/100
ask = value*100.03/100

spread = value*0.06/100 + value*0.03/100
spread = 0.09*value/100

skew = -0*1=0
skew = -1
skew = -2
skew = -3
skew = -4 (5th Position)

In order 

buy_quote (nth position) = 99.91*value/100 + skew(n_position)
sell_quote (nth position) = 100.09*value/100 + skew(n_position)

"""



'\nvalue = (bid+ask) /2\nspread = (ask-bid)+profit_target\nskew = -position*risk_adjustment\n\n\nbuy_quote=value+skew - spread\nsell_quote = value + skew + spread\n\n\nIn this problem,\n\nbid = value*99.97/100\nask = value*100.03/100\n\nspread = value*0.06/100 + value*0.03/100\nspread = 0.09*value/100\n\nskew = -0*1=0\nskew = -1\nskew = -2\nskew = -3\nskew = -4 (5th Position)\n\nIn order \n\nbuy_quote (nth position) = 99.91*value/100 + skew(n_position)\nsell_quote (nth position) = 100.09*value/100 + skew(n_position)\n\n'

In [100]:
df['bid_quote']=df['price']*99.97/100
df['ask_quote']=df['price']*100.03/100

In [101]:
df.drop(df[df['quoteQty']<1000].index,inplace=True)

# Final DataFrame File

In [102]:
df

,trade Id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch,bid_quote,ask_quote
8,1274557551,43160.01,0.02358,1017.713036,1646092800010,False,True,43147.061997,43172.958003
9,1274557552,43160.01,0.02329,1005.196633,1646092800010,False,True,43147.061997,43172.958003
12,1274557555,43160.00,0.04072,1757.475200,1646092800045,True,True,43147.052000,43172.948000
22,1274557565,43156.43,0.02362,1019.354877,1646092800045,True,True,43143.483071,43169.376929
24,1274557567,43156.15,0.02321,1001.654242,1646092800045,True,True,43143.203155,43169.096845
...,...,...,...,...,...,...,...,...,...
2871273,1277428816,44734.74,0.03000,1342.042200,1646233096446,False,True,44721.319578,44748.160422
2871274,1277428817,44734.74,0.02864,1281.202954,1646233096450,False,True,44721.319578,44748.160422
2871280,1277428823,44736.04,0.08083,3616.014113,1646233096536,False,True,44722.619188,44749.460812
2871281,1277428824,44736.04,0.08864,3965.402586,1646233096536,False,True,44722.619188,44749.460812


In [103]:
buy_orders =[43147.061997]
sell_orders=[43172.958003]

In [119]:
def logic():

    Net_Position=0 #For Calculation of Hedging Purposes
    buy_orders =[43147.061997] # Bid_Order_Book
    sell_orders=[43172.958003] # Ask_Order_Book

    buying_trades=[] #Prices at which Btc was bought @ 1000USD 
    selling_trades=[] #Prices at which Btc was sold @ 1000USD 
    
    total_trades=0 #Total Trades Done
    for i in range(1,len(df['price'])):
        
        if (df['price'].iloc[i]<= buy_orders[-1] and Net_Position<4):  #If the current price goes lower than Bid_Order_Book
            total_trades= total_trades+1
            
            Net_Position+=1
            buying_trades.append(buy_orders[-1]) #Adding the Buying Trades Positions
            buy_orders.pop(-1)
            
            if (len(sell_orders)<5):
                
                buy_orders.insert(0,df['bid_quote'].iloc[i])    #Modifying the bids and asks
                sell_orders.insert(0,df['ask_quote'].iloc[i])
                
            
            else:
                buy_orders.insert(0,df['bid_quote'].iloc[i])     #When 4000 usd trades are taken place on the side of buying

                buying_trades.append(sell_orders[-1])            #It starts Buying the farthest bids
                sell_orders.pop(-1)
                Net_Position+=1

                
                sell_orders.insert(0,df['ask_quote'].iloc[i])

            #print('Bid_Size =',buy_orders)
            #print('Ask_Size =',sell_orders)
            #print("LTP =",df['price'].iloc[i])
            #print(".......................")

        elif (df['price'].iloc[i]>= sell_orders[0] and Net_Position>-4):
            total_trades= total_trades+1
            selling_trades.append(sell_orders[0])
            Net_Position-=1
            sell_orders.pop(0)
            if (len(buy_orders)<5):
                
                buy_orders.append(df['bid_quote'].iloc[i])
                sell_orders.append(df['ask_quote'].iloc[i])
            
            
            else:    
                sell_orders.append(df['ask_quote'].iloc[i])
                
                selling_trades.append(buy_orders[0])
                buy_orders.pop(0)
                Net_Position-=1

                buy_orders.append(df['bid_quote'].iloc[i])


            #print('Bid_Size =',buy_orders)
            #print('Ask_Size =',sell_orders)
            #print("LTP =",df['price'].iloc[i])
            #print(".......................")


            

        
          
                


    return total_trades,buying_trades,selling_trades

In [120]:
Number_of_Trades, buying_trades, selling_trades=logic()

In [121]:
len(selling_trades)-len(buying_trades)

4

In [122]:
data = {'Buying':buying_trades,
        'Selling': selling_trades[:len(buying_trades)]}


In [123]:
result = pd.DataFrame(data,columns=['Buying','Selling'])

In [124]:
result['diff']=result['Selling']-result['Buying']

In [125]:
sum(result['diff'])

-16791.497065999814